In [ ]:
!pip install --upgrade numpy
!pip install --upgrade numexpr

In [ ]:
import os
import json
import traceback
import urllib.parse
import boto3
from datetime import datetime
import time
from smart_search import SmartSearchQA

In [ ]:
#根据时间情况修改index和language值
index =  "smart_search_qa_test_0601"
language = "chinese"

EMBEDDING_ENDPOINT_NAME = "huggingface-inference-eb"
port = 443
bulk_size = 10000000


sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-host-url')['SecretString']
data= json.loads(master_user)
es_host_name = data.get('host')
host = es_host_name+'/' if es_host_name[-1] != '/' else es_host_name# cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com/
host = host[8:-1]
region = boto3.Session().region_name # e.g. cn-north-1
print('host:',host)
print('region:',region)

# retrieve secret manager value by key using boto3                                             
sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-master-user')['SecretString']
data= json.loads(master_user)
username = data.get('username')
password = data.get('password')


host: search-smartsearch-o3ti34kef6ujmaljpm3asr65oi.us-east-2.es.amazonaws.com
region: us-east-2


In [ ]:
if language == "english":
    import nltk
    nltk.download('punkt')

search_qa = SmartSearchQA()
search_qa.init_cfg(index,
                 username,
                 password,
                 host,
                 port,
                 EMBEDDING_ENDPOINT_NAME,
                 region,
                 language=language
                 )

local_path = "../test_file/"

files = os.listdir(local_path)
for file in files:
    local_file = local_path+file
    now1 = datetime.now()#begin time
    loaded_files = search_qa.init_knowledge_vector(local_file,bulk_size)
    now2 = datetime.now()#endtime
    print("File import takes time:",now2-now1)
    print("Complete the import of the following documents:", str(loaded_files))



基于预训练语言模型的行业搜索的应用和研究.docx Loaded successfully
The file is loaded and the vector library is being generated
finish embedding, text len: 116 116
File import takes time: 0:00:06.965437
Complete the import of the following documents: ['../test_file/基于预训练语言模型的行业搜索的应用和研究.docx']
